In [45]:
#import os
import math
import numpy as np
import pandas as pd
import scipy.io
from pathlib import Path
#import numba

from mayavi import mlab
import quaternion as quat
from sklearn.decomposition import PCA

# Libraries
See environment.yml 

conda env create -f environment.yml

conda activate sci


## OS
Has been written (and runs) on both Windows 10 and MacOS

## Python
This was written on python 3.6, python 2.7x versions won't work due to the f strings

## mayavi
This is the 3d plotting library used for rendering the plots. mayvai will launch a qt window to display the plot- meaning that you will need an X serve session for the plots to load. If you want to plot things inline you will need to use jupyter notebooks, not jupyter lab.

### mayavi install
https://docs.enthought.com/mayavi/mayavi/installation.html#installing-with-conda-forge


    
## numpy-quaternion
https://github.com/moble/quaternion
https://quaternion.readthedocs.io/en/latest/

    conda install -c conda-forge quaternion
    
    
## Optional: inline 3d plotting
http://docs.enthought.com/mayavi/mayavi/tips.html#using-mayavi-in-jupyter-notebooks

# Classes

This class manages the loading of voxel objects and creating its atributes 

## bone


In [71]:
class bone:

    filter_level = 0.001
    default_color = (0.7, 1, 1)
    scale = 1.5

    def __init__(self, array):
        """
        Performs calculations on the voxel array objects    
        array (np.array): binary voxel object)      
        filter_level (int/float): sets the threshold level for 
        what is considered a voxel. Everything below filter level is
        rounded to 0, everything above rounded to 1 (ie voxel)
        """
        self.array = array
        self.get_xyz()

    def get_xyz(self):
        """Convert 3D voxel array to xyz coordinates.

        array (np.array): 3D voxel array  

        filter_level (int/float): (inherited from `bone` class) sets the threshold level for 
        what is considered a voxel. Everything below filter level is
        rounded to 0, everything above rounded to 1 (ie voxel)

        returns: 
            np.array( [n x 3] )"""

        # Everything above filter level is converted to 1
        filtered_array = np.where(self.array < self.filter_level, 0, 1)

        # records coordiates where there is a 1
        x, y, z = np.where(filtered_array == 1)

        self.xyz = np.array([x, y, z]).T

    def get_pca(self):
        """PCA on the xyz points array

            xyz(np.array): n x 3 array of xyz coordinates

            returns:    self.pc1
                        self.pc2
                        self.pc3"""

        pca = PCA(svd_solver='full')
        pca.fit(self.xyz)
         
        self.list = pca.components_
        self.pc1 = pca.components_[0]
        self.pc2 = pca.components_[1]
        self.pc3 = pca.components_[2]


    def get_mean(self):
        """The mean of the xyz atriube

            returns:
                tupple (mean_of_x, mean_of_y ,mean_of_z)"""

        #mean_x, mean_y, mean_z
        return (np.mean(self.xyz[:, 0]), np.mean(self.xyz[:, 1]), np.mean(self.xyz[:, 2]))

    def center_to_origin(self):
        """ sets the mean of the bone to 0,0,0"""

        # set transformation (tfm) value
        self.tfm = self.get_mean()

        self.xyz = self.xyz - self.tfm

    def reset_position(self):
        """ resets the position of the bone to its orginal one"""
        self.xyz = self.xyz + self.tfm


    def plot(self, user_color=None, PCA_inv=None, PCA=True):
        """ Plot voxels with optional PCA, and colours
        
            user_color (tupple): RGB color of the bone where 1 is maxium
                                    eg: red = (1,0,0)
                                    
            PCA (boolean): plots the PCAs of the voxel
            
            PCA_inv (boolean): plots the inverse of each PCA so the axes go in both directions
        """
                
        if hasattr(self, 'pc1') is False:
            self.get_pca()
        
        x, y, z = self.get_mean()
        
        if user_color is None:
            user_color = self.default_color
    
        #plots voxels 
        mlab.points3d(self.xyz[:, 0],
                      self.xyz[:, 1],
                      self.xyz[:, 2],
                      mode = "cube",
                      color= user_color,
                      scale_factor = 1)   
        
        # plots pca arrows
        if PCA is True:
        
            mlab.quiver3d(x, y, z, *self.pc1,
                              line_width=6,
                              scale_factor=100,
                              color=(1, 0, 0))
            mlab.quiver3d(x, y, z, *self.pc2,
                              line_width=6,
                              scale_factor=50,
                              color=(0, 1, 0))
            mlab.quiver3d(x, y, z, *self.pc3,
                              line_width=6,
                              scale_factor=30,
                              color=(0, 0, 1))
        
        
       # plots the pca *-1  
        if PCA_inv is True:
        
            mlab.quiver3d(x, y, z, *(self.pc1*-1),
                              line_width=6,
                              scale_factor=100,
                              color=(1, 0, 0))
            mlab.quiver3d(x, y, z, *(self.pc2*-1),
                              line_width=6,
                              scale_factor=50,
                              color=(0, 1, 0))
            mlab.quiver3d(x, y, z, *(self.pc3*-1),
                              line_width=6,
                              scale_factor=30,
                              color=(0, 0, 1))
            


    def upscale(self):
        """ upscales the bone """
        n = 1

        self.center_to_origin()
        
        while n < self.scale:

            n += 0.1

            scalled = self.xyz * np.array([n, n, n])
            self.xyz = np.concatenate((scalled, self.xyz))

            print(f"scale = {n}")

            # if xyz.shape > max_lim:
            #     brake
            #     print('xyz point limit hit')

    def downscale(self):
        """ scales bone back to orginal dementions"""

        self.xyz = self.xyz /np.array([self.scale,self.scale,self.scale])

    #     Alternative constructor:
    #     Import directly from matlab path

    @classmethod
    def from_matlab_path(cls, matlab_file):
        """Imports matlab file drectly

           path: path object/string 

           retruns np.array (n x n x n )"""

        matlab_object = scipy.io.loadmat(matlab_file)
        obj = matlab_object.keys()
        obj = list(obj)
        array = matlab_object[obj[-1]]

        return cls(array)   

# Maths functions

## mag

In [48]:
def mag(v):
    """ Finds magnitude of vector

        v (np.array): vector"""
    return math.sqrt(np.dot(v, v))

## angle

https://uk.mathworks.com/matlabcentral/answers/101590-how-can-i-determine-the-angle-between-two-vectors-in-matlab

In [49]:
def angle(v1, v2):
    """ Finds angel between 2 vectors

    returns: ang , v1"""
    
    try:
       
        ang = math.atan2(np.linalg.norm(np.cross(v1,v2)),np.dot(v1,v2))
                         
        if ang > math.pi/2:
            v1 = -v1
            ang = math.atan2(np.linalg.norm(np.cross(v1,v2)),np.dot(v1,v2))
            
            print(f'{ang} PC inverted')

        else:
            print(f'{ang} no invert')

    except:
        #vang = 0
        print(f'ERROR: vectors v1= {v1}, v2= {v2}')
        ang = 'ERROR'

    return ang, v1


# Rotation

The bones are rotated with quaternions.

The angle between the two PC1 vectors is taken. The object is then rotated (by a quaternion) around the cross product between the PC1 vectors.

The new angles between the next PCs are calculates and the process is repeated for the other PCs

## quaternion_rotation

In [50]:
def quaternion_rotation_from_angle(v, c_axis, theta):

    rotation_axis = np.array([0.] + c_axis)
    axis_angle = (theta*0.5) * rotation_axis/np.linalg.norm(rotation_axis)

    vec = quat.quaternion(*v)

    # quaternion from exp of axis angle
    qlog = quat.quaternion(*axis_angle)
    q = np.exp(qlog)

    # double cover quaternion rotation
    v_prime = q * vec * np.conjugate(q)

    return v_prime.imag 

## voxel_rotate

In [51]:
def voxel_rotate(bone_f1, bone_f2):

    quaterion_product = None
    
    # center bones too 0,0,0,
    bone_f1.center_to_origin()
    bone_f2.center_to_origin()

    # PCA on bones
    bone_f1.get_pca()
    bone_f2.get_pca()

    # for 1 to 3 principle conponents of the object
    for n in range(1, 4):

        # takes cross product axis
        cross_product_axis = np.cross(
            getattr(bone_f1, f'pc{n}'),
            getattr(bone_f2, f'pc{n}'))

        # finds angle between PCs for f1 vs f2
        theta, vector = angle(
            getattr(bone_f1, f'pc{n}'),
            getattr(bone_f2, f'pc{n}'))

        # sets any new values needed
        setattr(bone_f1, f'pc{n}', vector)

        # rotates each PC
        for n in range(1, 4):
            transformed_pc = quaternion_rotation_from_angle(
                v=getattr(bone_f1, f'pc{n}'),
                c_axis=cross_product_axis,
                theta=theta)

            # sets new PCA
            setattr(bone_f1, f'pc{n}', transformed_pc)


        # rotates xyz array with the quaterion product
        rotated_xyz = np.apply_along_axis(
            quaternion_rotation_from_angle, 1, 
            getattr(bone_f1, 'xyz'),
            c_axis=cross_product_axis,
            theta=theta)

        setattr(bone_f1, 'xyz', rotated_xyz)

        bone_f1.out = quaterion_product

# Table of Angles

## df_angles

In [52]:
def df_angles(bone_f1, bone_f2, name='UN-NAMED BONE'):
    """
    Compares the PCA angles between to bones.

    Input:  bone_f1 = bone in 1st position
            bone_f2 = bone in 2nd position

    Returns: pandas dataframe
    """

    df = pd.DataFrame()

    
    # Check for PCAs
    if hasattr(bone_f1, 'pc1') is False:
            bone_f1.get_pca()
    
    if hasattr(bone_f2, 'pc1') is False:
            bone_f2.get_pca()
            
    # loops over each PCA
    for n in range(1, 4):
        theta, _ = angle(
            getattr(bone_f1, f'pc{n}'), getattr(bone_f2, f'pc{n}'))

        # Sets the column names
        df.loc[f'{name} f1: pc{n}', f'{name} f2: pc{n}'] = theta

    return df

# How to use:

## 1. Load the data that you want to use
*you have to use the absoslute file paths*

In [72]:
# load data phantoms
tibia_f2 = bone.from_matlab_path(
    matlab_file='/home/luke/phantom/phantom_tibia_f2.mat')

tibia_f1 = bone.from_matlab_path(
    matlab_file='/home/luke/phantom/phantom_tibia_f1.mat')

### *Set custom filter level (optional)*

In [73]:
#bone.filter_level = 0.1

### *Set custom colour for bone(optional)*

In [74]:
tibia_f1.default_color = (0.8, 0.3, 0)

## Table of Angles

The angles between the PCAs

In [39]:
df_angles(tibia_f1, tibia_f2, name='tibia')

0.1747168069985996 no invert
0.5798220207976942 no invert
0.6067394097718334 no invert


,tibia f2: pc1,tibia f2: pc2,tibia f2: pc3
tibia f1: pc1,0.174717,NaN,NaN
tibia f1: pc2,NaN,0.579822,NaN
tibia f1: pc3,NaN,NaN,0.606739


## Rotate the Bone

In [58]:
voxel_rotate(tibia_f1, tibia_f2)

0.17471680699860007 no invert
0.5819453559235347 no invert
3.906788298496456e-16 no invert


## Plotting the Rotation

In [75]:
tibia_f1.plot()
tibia_f2.plot()
mlab.show()

In [68]:
tibia_f1.upscale()

scale = 1.1
scale = 1.2000000000000002
scale = 1.3000000000000003
scale = 1.4000000000000004
scale = 1.5000000000000004


In [61]:
tibia_f1.xyz.shape

(695360, 3)

In [70]:
tibia_f1.plot()
mlab.show()

In [69]:
tibia_f1.downscale()

In [64]:
tibia_f1.plot()
mlab.show()